# Lecture 3. Decision Trees 

## Theory

Let's look at a toy example. We will predict the color of the ball based on its position

#### Entropy
Shannon's entropy is defined for a system with N possible states as follows:

$$\Large S = -\sum_{i=1}^{N}p_i \log_2{p_i},$$

where $p_i$ is the probability of finding the system in the $i$-th state. This is a very important concept used in physics, information theory, and other areas. Entropy can be described as the degree of chaos in the system. The higher the entropy, the less ordered the system

<img src='../data/decision_tree3.png'><br>

There are 9 blue balls and 11 yellow balls. If we randomly pull out a ball, then it will be blue with probability $p_1=\frac{9}{20}$ and yellow with probability $p_2=\frac{11}{20}$, which gives us an entropy $S_0 = -\frac{9}{20}\log_2{\frac{9}{20}}-\frac{11}{20}\log_2{\frac{11}{20}} \approx 1$. This value by itself may not tell us much, but let's see how the value changes if we were to break the balls into two groups: with the position less than or equal to 12 and greater than 12.

<img src='../data/decision_tree4.png'><br>

The left group has 13 balls, 8 blue and 5 yellow. The entropy of this group is $S_1 = -\frac{5}{13}\log_2{\frac{5}{13}}-\frac{8}{13}\log_2{\frac{8}{13}} \approx 0.96$. The right group has 7 balls, 1 blue and 6 yellow. The entropy of the right group is $S_2 = -\frac{1}{7}\log_2{\frac{1}{7}}-\frac{6}{7}\log_2{\frac{6}{7}} \approx 0.6$. As you can see, entropy has decreased in both groups, more so in the right group. Since entropy is, in fact, the degree of chaos (or uncertainty) in the system, the reduction in entropy is called information gain. Formally, the information gain (IG) for a split based on the variable $Q$ (in this example it's a variable "$x \leq 12$") is defined as

$$\Large IG(Q) = S_O - \sum_{i=1}^{q}\frac{N_i}{N}S_i,$$

where  𝑞  is the number of groups after the split,  𝑁𝑖  is number of objects from the sample in which variable  𝑄  is equal to the  𝑖 -th value. In our example, our split yielded two groups ( 𝑞=2 ), one with 13 elements ( 𝑁1=13 ), the other with 7 ( 𝑁2=7 ). Therefore, we can compute the information gain as

$$ \Large IG(x \leq 12) = S_0 - \frac{13}{20}S_1 - \frac{7}{20}S_2 \approx 0.16.$$

It turns out that dividing the balls into two groups by splitting on "coordinate is less than or equal to 12" gave us a more ordered system. Let's continue to divide them into groups until the balls in each group are all of the same color.

<img src='../data/decision_tree5.png'><br>

For the right group, we can easily see that we only need one extra partition using "coordinate less than or equal to 18". But, for the left group, we need three more. Note that the entropy of a group where all of the balls are the same color is equal to 0 ($\log_2{1} = 0$).

We have successfully constructed a decision tree that predicts ball color based on its position. This decision tree may not work well if we add any balls because it has perfectly fit to the training set (initial 20 balls). If we wanted to do well in that case, a tree with fewer "questions" or splits would be more accurate, even if it does not perfectly fit the training set. (info from MLCourse Open) 

## Let's create our first decision tree

In [2]:
# At first we have to install sklearn library
#!pip3 install sklearn

In [3]:
#import necessary libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

Read a dataset from previous lecture

In [4]:
data = pd.read_csv('../data/telecom_churn.csv')

In [5]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


It will be better to cast all features to a numeric type and to drop some useless features

In [6]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace = True)

In [7]:
data['International plan'] = data['International plan'].map({'Yes':1, 'No':0})

Now all features are numeric and the target feature is boolean

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   int64  
 3   Number vmail messages   3333 non-null   int64  
 4   Total day minutes       3333 non-null   float64
 5   Total day calls         3333 non-null   int64  
 6   Total day charge        3333 non-null   float64
 7   Total eve minutes       3333 non-null   float64
 8   Total eve calls         3333 non-null   int64  
 9   Total eve charge        3333 non-null   float64
 10  Total night minutes     3333 non-null   float64
 11  Total night calls       3333 non-null   int64  
 12  Total night charge      3333 non-null   float64
 13  Total intl minutes      3333 non-null   float64
 14  Total intl calls        3333 non-null   

Let's prepare the data for training

In [9]:
#cast boolean to int
y = data['Churn'].astype('int')

and drop the target feature from the training x-dataset

In [10]:
x = data.drop('Churn', axis = 1)

In [11]:
x.shape , y.shape

((3333, 17), (3333,))

Then we split the training set into Train and Test part via sklearn library method

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(x,y,test_size=0.3, random_state=17)

In [14]:
X_train.shape, X_valid.shape

((2333, 17), (1000, 17))

Let's create our first tree instance

In [15]:
first_tree = DecisionTreeClassifier(random_state=17)

## Cross Validation

In Machine learning, we usually divide the dataset into Training dataset, Validation dataset, and Test dataset

<img src='../data/dataset1.png'><br>

- Training data set — used to train the model, it can vary but typically we use 60% of the available data for training.
- Validation data set — Once we select the model that performs well on training data, we run the model on validation data set. This is a subset of the data usually ranges from 10% to 20%. Validation data set helps provide an unbiased evaluation of the model’s fitness. If the error on the validation dataset increases then we have an overfitting model.
- Test dataset — Also called as holdout data set. This dataset contains data that has never been used in the training. Test data set helps with final model evaluation. Typically would be 5% to 20% of the dataset.

Sometimes there can be only training and test set and no validation set.

### What’s the problem with this approach?
- Due to sample variability between training and test set, our model gives a better prediction on training data but fail to generalize on test data. This leads to a low training error rate but a high test error rate.
- When we split the dataset into training, validation and test set, we use only a subset of data and we know when we train on fewer observations the model will not perform well and overestimate the test error rate for the model to fit on the entire dataset

### To solve the two issue we use an approach called cross-validation
Cross-validation is a statistical technique which involves partitioning the data into subsets, training the data on a subset and use the other subset to evaluate the model’s performance.

One useful type of Cross Validation(CV) is K fold cross validation

## K fold cross validation

This technique involves randomly dividing the dataset into k groups or folds of approximately equal size. The first fold is kept for testing and the model is trained on k-1 folds.
The process is repeated K times and each time different fold or a different group of data points are used for validation.

<img align='center' src='../data/cross_validation.png'><br>

orange block is the fold used for testing

let's do it practically

In [16]:
from sklearn.model_selection import cross_val_score

In [17]:
cross_val_score(first_tree, X_train, y_train, cv=5)

array([0.9143469 , 0.91220557, 0.92077088, 0.90772532, 0.91416309])

Mean result for all dataset:

In [18]:
import numpy as np

In [19]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9138423504976518

One of the most important hyperparameter is maximum depth for tree, let's' try to change it with GridSearch instrument, which allows us to choose the most optimal value automatically




In [20]:
from sklearn.model_selection import GridSearchCV  

In [21]:
tree_params = {'max_depth':np.arange(1,10)}

max depth will be from 1 to 9

In [22]:
np.arange(1,10)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [23]:
tree_grid = GridSearchCV(first_tree, tree_params, cv = 5, n_jobs=4)

Let's train GridSearchCV using our dataset

In [24]:
%time
tree_grid.fit(X_train, y_train);

CPU times: total: 0 ns
Wall time: 0 ns


The best result will be

In [25]:
tree_grid.best_score_, tree_grid.best_params_

(0.9417108564391468, {'max_depth': 6})

In [26]:
tree_grid.best_estimator_

DecisionTreeClassifier(max_depth=6, random_state=17)

And predict the result on the validation set

In [27]:
tree_valid_pred = tree_grid.predict(X_valid)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
accuracy_score(y_valid, tree_valid_pred)

0.946

## Visualization

For visualization, we use Graphviz module. You have to import it from Sklearn 

In [30]:
from sklearn.tree import export_graphviz

Then export our tree

In [31]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot',
                feature_names=x.columns, filled = True, max_depth=3)

In [32]:
!ls -l

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


To convert .dot file to .png format we should use Pydot library, let's install it

Convert the tree

In [33]:
!dot -Tpng telecom_tree.dot -o telecom_tree.png

'dot' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [34]:
import pydot

In [42]:
import pydot

(graph,) = pydot.graph_from_dot_file('telecom_tree.dot')


In [43]:
graph.write_png('telecom_tree.png')

FileNotFoundError: [WinError 2] "dot" not found in path.

and visualize the picture in notebook

<img src='telecom_tree.png'>

Also, we can create a simple decision tree without boosting hyperparameters. For example, a decision tree with max_depth=3

In [44]:
second_tree = DecisionTreeClassifier(max_depth =3).fit(X_train, y_train)


In [45]:
second_tree.score(X_valid, y_valid)

0.905

In [46]:
export_graphviz(second_tree, out_file='telecom_tree_second.dot',
                feature_names=x.columns, filled = True,max_depth=3)

In [47]:
!ls

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [48]:
!dot -Tpng telecom_tree_second.dot -o telecom_tree_second.png

'dot' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


<img src='telecom_tree_second.png'>